In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

import os
import numpy as np
np.warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
from tqdm import tqdm

from google_drive_downloader import GoogleDriveDownloader as gdd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.contrib.eager.python import tfe
from PIL import Image

tf.enable_eager_execution()
tf.set_random_seed(0)
np.random.seed(0)

Using TensorFlow backend.


In [0]:
gdd.download_file_from_google_drive(file_id='1ycR7Aexe8xbZ8oEDsQwGc9SIiFklRpfu', dest_path='./data.zip', unzip=True)

Unzipping...Done.


In [0]:
data_dir = 'data'
os.listdir(data_dir)

['sample_submission.csv', 'train.csv', 'images']

In [0]:
#Read data
train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
train_df.head()

,image,label
0,VietAI-Assignment3-1.jpg,7
1,VietAI-Assignment3-100.jpg,2
2,VietAI-Assignment3-10000.jpg,1
3,VietAI-Assignment3-10001.jpg,2
4,VietAI-Assignment3-10002.jpg,2


In [0]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8234 entries, 0 to 8233
Data columns (total 2 columns):
image    8234 non-null object
label    8234 non-null int64
dtypes: int64(1), object(1)
memory usage: 128.7+ KB


In [0]:
test_df = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
test_df.head()

,image,label
0,VietAI-Assignment3-10.jpg,0
1,VietAI-Assignment3-1000.jpg,0
2,VietAI-Assignment3-10004.jpg,0
3,VietAI-Assignment3-10006.jpg,0
4,VietAI-Assignment3-10012.jpg,0


In [0]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 2 columns):
image    2059 non-null object
label    2059 non-null int64
dtypes: int64(1), object(1)
memory usage: 32.2+ KB


In [0]:
train_df.label.value_counts()

2     1949
1      846
0      808
3      764
5      642
4      599
7      579
10     575
6      535
8      469
9      468
Name: label, dtype: int64

##TODO 1: Cài đặt hàm đọc ảnh và đưa về  NumPy Array

In [0]:
#generate_data to create train and test matrix
def generate_data(image_paths, size=224):
    """
    Đọc và chuyển các ảnh về numpy array
    
    Parameters
    ----------
    image_paths: list of N strings
        List các đường dẫn ảnh
    size: int
        Kích thước ảnh cần resize
    
    Returns
    -------
    numpy array kích thước (N, size, size, 3)
    """
    image_array = np.zeros((len(image_paths), size, size, 3), dtype='float32')
    
    for idx, image_path in tqdm(enumerate(image_paths)):
        ### START CODE HERE
        image = Image.open(image_path)
        image = image.resize((size,size))
        
        # Chuyển ảnh thành numpy array và gán lại mảng image_array
        img = np.asarray(image)
        
        #normalize
        # img = img/255
        
        image_array[idx] = img
        ### END CODE HERE
    return image_array

In [0]:
# List các đường dẫn file cho việc huấn luyện
train_files = [os.path.join("data/images", file) for file in train_df.image]

# List các nhãn
train_y = train_df.label

# Tạo numpy array cho dữ liệu huấn luyện
train_arr = generate_data(train_files)

8234it [00:32, 251.28it/s]


In [0]:
train_arr.shape
# train_arr[1]
# train_y.shape

(8234, 224, 224, 3)

In [0]:
#Tao tensor cho du lieu test

test_files = [os.path.join("data/images", file) for file in test_df.image]
test_x = generate_data(test_files)
test_x.shape

2059it [00:08, 252.72it/s]


(2059, 224, 224, 3)

In [0]:
# test_x[:5]

In [0]:
#Tao one-hot cho train_y

num_classes = len(np.unique(train_y))
y_ohe = tf.keras.utils.to_categorical(train_y, num_classes=num_classes)

## Chia dữ liệu để huấn luyện và đánh giá

In [0]:
x_train, x_valid, y_train_ohe, y_valid_ohe = train_test_split(train_arr, y_ohe, test_size=0.25)

print("Train size: {} - Validation size: {}".format(x_train.shape, x_valid.shape))

Train size: (6175, 224, 224, 3) - Validation size: (2059, 224, 224, 3)


## Xây dựng mô hình

In [0]:
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [0]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'w'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'w'),
            bottleneck_features_validation)

In [0]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy'))
    train_labels = np.array(
        [0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))

    validation_data = np.load(open('bottleneck_features_validation.npy'))
    validation_labels = np.array(
        [0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

## Huấn luyện

In [19]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
 
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate
 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)


170500096/170498071 [==============================] - 2s 0us/step


In [25]:
x_test.shape

(10000, 32, 32, 3)

In [0]:
 
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()
 
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)
 
#training
batch_size = 64
 
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])
#save to disk
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5') 
 
#testing
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0])

## Dự đoán các ảnh trên tập Test

### Tạo và load model đã lưu trước đó

In [0]:
# Load best model
model = CNN(num_classes)

# Thiết lập kích thước input cho model
dummy_x = tf.zeros((1, 224, 224, 3))
model._set_inputs(dummy_x)

# Load model đã lưu trước đó trong quá trình huấn luyện
model.load_weights('my_model.h5')
print("Model đã được load")

### Dự đoán nhãn của các ảnh trên tập Test

In [0]:
pred = model.predict(test_x)

# pred là một ma trận xác suất của ảnh trên các lớp.
# Ta lấy lớp có xác suất cao nhất trên từng ảnh bằng hàm argmax
pred_labels = np.argmax(pred, axis=1)

In [0]:
test_df['label'] = pred_labels
test_df.head(20)

#### Lưu kết quả thành file CSV

In [0]:
test_df.to_csv("submission.csv", index=False)

In [0]:
device = '/cpu:0' if tfe.num_gpus() == 0 else '/gpu:0'
batch_size = 32
epochs = 16

with tf.device(device):
    # Khởi tạo model
    model = CNN(num_classes)
    
    # Tạo callback để lưu model có accuracy trên tập validation tốt nhất
    mcp = tf.keras.callbacks.ModelCheckpoint("my_model.h5", monitor="val_acc",
                      save_best_only=True, save_weights_only=True)
    
    # Compile model
    model.compile(optimizer=tf.train.AdamOptimizer(0.001), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Huấn luyện
    model.fit(x_train, y_train_ohe, batch_size=batch_size, epochs=epochs,
              validation_data=(x_valid, y_valid_ohe), verbose=1, callbacks=[mcp])
